In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn import pipeline
from sklearn.feature_selection import SelectKBest, f_regression, f_classif, chi2, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from scipy.stats import pearsonr
from sklearn.metrics import confusion_matrix, classification_report, silhouette_score, f1_score, precision_recall_curve, plot_precision_recall_curve, average_precision_score, auc
from sklearn.svm import SVR, LinearSVR, SVC, LinearSVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from scipy import stats
from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import plot_confusion_matrix

In [ ]:
year = "2019"
IN = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/Household_Indicators_Input/household_input_'+year+'_combined_new.pickle', 'rb'))
X=IN.copy()
IN

,BF_0_OUT_1,BF_0_OUT_2,BF_0_OUT_3,FC_0_OUT_1,FC_0_OUT_2,FC_0_OUT_3,MSW_0_OUT_1,MSW_0_OUT_2,MSW_0_OUT_3,ASSET_0_OUT_1,ASSET_0_OUT_2,ASSET_0_OUT_3,LIT_0_OUT_1,LIT_0_OUT_2,LIT_0_OUT_3,BF_1_OUT_1,BF_1_OUT_2,BF_1_OUT_3,FC_1_OUT_1,FC_1_OUT_2,FC_1_OUT_3,MSW_1_OUT_1,MSW_1_OUT_2,MSW_1_OUT_3,ASSET_1_OUT_1,ASSET_1_OUT_2,ASSET_1_OUT_3,LIT_1_OUT_1,LIT_1_OUT_2,LIT_1_OUT_3,BF_2_OUT_1,BF_2_OUT_2,BF_2_OUT_3,FC_2_OUT_1,FC_2_OUT_2,FC_2_OUT_3,MSW_2_OUT_1,MSW_2_OUT_2,MSW_2_OUT_3,ASSET_2_OUT_1,...,BF_N_OUT_3,FC_N_OUT_1,FC_N_OUT_2,FC_N_OUT_3,MSW_N_OUT_1,MSW_N_OUT_2,MSW_N_OUT_3,ASSET_N_OUT_1,ASSET_N_OUT_2,ASSET_N_OUT_3,LIT_N_OUT_1,LIT_N_OUT_2,LIT_N_OUT_3,dc_score_none_none,dc_score_none_size,dc_score_none_size_intensity,dc_score_none_intensity,dc_score_log_none,dc_score_log_size,dc_score_log_size_intensity,dc_score_log_intensity,dc_score_sqrt_none,dc_score_sqrt_size,dc_score_sqrt_size_intensity,dc_score_sqrt_intensity,dc_score_none_size_only,dc_score_none_size_intensity_only,dc_score_none_intensity_only,dc_score_log_size_only,dc_score_log_size_intensity_only,dc_score_log_intensity_only,dc_score_sqrt_size_only,dc_score_sqrt_size_intensity_only,dc_score_sqrt_intensity_only,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
27378,0.968222,0.017454,1.432456e-02,0.641587,0.002096,0.356317,0.344000,0.138653,0.517347,0.248841,0.255712,0.495447,0.010680,0.154920,0.834400,0.002592,2.194939e-02,9.754584e-01,0.021356,0.000162,0.978482,0.151773,0.130277,0.717949,0.000008,0.000184,0.999808,0.038358,0.195534,0.766109,0.939563,0.000722,0.059715,0.719036,0.000077,0.280887,0.105428,0.001011,0.893561,0.012313,...,0.124662,0.249257,0.005061,0.745682,0.067736,0.031364,0.900899,0.004492,0.078104,0.917404,0.008692,0.199259,0.792049,0.159114,-0.213076,-0.321962,-0.226048,0.907909,-0.314416,-0.191585,-0.261134,0.625911,-0.353732,-0.300795,-0.335504,-1.436431,0.963518,-0.677461,-1.529542,0.784563,-1.228502,-1.610552,0.903286,-0.907379,-0.010717,-0.018753,-0.127615,-0.277275,0.109255,-0.116993
27379,0.050541,0.816297,1.331623e-01,0.171801,0.000346,0.827853,0.005832,0.014344,0.979824,0.007810,0.145205,0.846985,0.006554,0.066524,0.926922,0.001948,9.819639e-01,1.608787e-02,0.164178,0.020870,0.814951,0.103190,0.020410,0.876400,0.000435,0.169562,0.830003,0.059343,0.315054,0.625603,0.064368,0.004776,0.930856,0.080722,0.001315,0.917963,0.000420,0.007279,0.992301,0.000501,...,0.206250,0.151487,0.012391,0.836123,0.010114,0.053958,0.935928,0.001630,0.022931,0.975439,0.030276,0.290913,0.678810,0.193251,-0.197112,-0.322261,-0.133677,0.983028,-0.228272,-0.192280,0.075560,0.703741,-0.301439,-0.301311,-0.094377,-1.436431,0.963518,-0.677461,-1.529542,0.784563,-1.228502,-1.610552,0.903286,-0.907379,-0.011752,-0.022067,-0.241814,-0.350502,0.006275,-0.199610
27380,0.939563,0.000722,5.971516e-02,0.719036,0.000077,0.280887,0.105428,0.001011,0.893561,0.012313,0.303302,0.684385,0.008113,0.337719,0.654168,0.002592,2.194939e-02,9.754584e-01,0.021356,0.000162,0.978482,0.151773,0.130277,0.717949,0.000008,0.000184,0.999808,0.038358,0.195534,0.766109,0.968222,0.017454,0.014325,0.641587,0.002096,0.356317,0.344000,0.138653,0.517347,0.248841,...,0.055414,0.235297,0.009466,0.755237,0.087799,0.085868,0.826334,0.008685,0.079988,0.911327,0.009081,0.168607,0.822312,0.223736,-0.220258,-0.317460,-0.336055,1.046677,-0.355077,-0.181193,-0.806979,0.771443,-0.377830,-0.293072,-0.670224,-1.436431,0.963518,-0.677461,-1.529542,0.784563,-1.228502,-1.610552,0.903286,-0.907379,0.007654,0.026277,0.889576,0.631396,0.926002,0.733766
27381,0.002592,0.021949,9.754584e-01,0.021356,0.000162,0.978482,0.151773,0.130277,0.717949,0.000008,0.000184,0.999808,0.038358,0.195534,0.766109,0.001948,9.819639e-01,1.608787e-02,0.164178,0.020870,0.814951,0.103190,0.020410,0.876400,0.000435,0.169562,0.830003,0.059343,0.315054,0.625603,0.968222,0.017454,0.014325,0.641587,0.002096,0.356317,0.344000,0.138653,0.517347,0.248841,...,0.051438,0.443485,0.007611,0.548904,0.083514,0.033244,0.883242,0.030855,0.284983,0.684162,0.006713,0.189698,0.803590,0.023075,-0.1999

In [ ]:
# cols_best = ['dc_score_sqrt_size_intensity', 'dc_score_log_size_intensity',
#        'dc_score_none_size_intensity', 'dc_score_none_size',
#        'dc_score_sqrt_size', 'dc_score_log_size', 'dc_score_none_size_only',
#        'dc_score_sqrt_size_intensity_only', 'dc_score_log_size_only',
#        'dc_score_sqrt_size_only', 'dc_score_none_size_intensity_only',
#        'MSW_N_OUT_2', 'dc_score_log_size_intensity_only', 'MSW_N_OUT_1',
#        'MSW_N_OUT_3', 'FC_N_OUT_1', 'FC_N_OUT_2',
#        'dc_score_none_intensity_only', 'ASSET_N_OUT_2',
#        'dc_score_sqrt_intensity_only', 'dc_score_log_intensity_only',
#        'dc_score_log_intensity', 'ASSET_N_OUT_1', 'dc_score_none_intensity',
#        'dc_score_sqrt_intensity', 'BF_N_OUT_1', 'LIT_N_OUT_2', 'LIT_N_OUT_3',
#        'MSW_0_OUT_1']

cols_best = ['log_hh', 'sqrt_hh', 'hh', 'log_pop', 'sqrt_pop', 'pop', 'MSW_N_OUT_3',
       'MSW_0_OUT_3', 'ASSET_N_OUT_3', 'MSW_N_OUT_2', 'FC_N_OUT_2',
       'dc_score_sqrt_size_intensity_only',
       'dc_score_none_size_intensity_only', 'dc_score_log_size_intensity_only',
       'MSW_4_OUT_3', 'MSW_0_OUT_2', 'MSW_1_OUT_3', 'LIT_N_OUT_3',
       'dc_score_none_size_only', 'dc_score_log_size_intensity',
       'dc_score_sqrt_size_intensity', 'dc_score_none_size_intensity',
       'MSW_N_OUT_1', 'FC_0_OUT_2', 'MSW_5_OUT_3', 'dc_score_sqrt_size_only',
       'dc_score_log_size_only', 'FC_3_OUT_2', 'ASSET_N_OUT_1',
       'dc_score_sqrt_size', 'MSW_2_OUT_3', 'dc_score_log_size', 'FC_1_OUT_2',
       'MSW_3_OUT_3', 'dc_score_none_size', 'MSW_5_OUT_2']


X11 = X.loc[:,cols_best]
X11

,log_hh,sqrt_hh,hh,log_pop,sqrt_pop,pop,MSW_N_OUT_3,MSW_0_OUT_3,ASSET_N_OUT_3,MSW_N_OUT_2,FC_N_OUT_2,dc_score_sqrt_size_intensity_only,dc_score_none_size_intensity_only,dc_score_log_size_intensity_only,MSW_4_OUT_3,MSW_0_OUT_2,MSW_1_OUT_3,LIT_N_OUT_3,dc_score_none_size_only,dc_score_log_size_intensity,dc_score_sqrt_size_intensity,dc_score_none_size_intensity,MSW_N_OUT_1,FC_0_OUT_2,MSW_5_OUT_3,dc_score_sqrt_size_only,dc_score_log_size_only,FC_3_OUT_2,ASSET_N_OUT_1,dc_score_sqrt_size,MSW_2_OUT_3,dc_score_log_size,FC_1_OUT_2,MSW_3_OUT_3,dc_score_none_size,MSW_5_OUT_2
27378,0.109255,-0.116993,-0.018753,-0.127615,-0.277275,-0.010717,0.900899,0.517347,0.917404,0.031364,0.005061,0.903286,0.963518,0.784563,0.820959,0.138653,0.717949,0.792049,-1.436431,-0.191585,-0.300795,-0.321962,0.067736,0.002096,0.923492,-1.610552,-1.529542,0.020870,0.004492,-0.353732,0.893561,-0.314416,0.000162,0.876400,-0.213076,0.053216
27379,0.006275,-0.199610,-0.022067,-0.241814,-0.350502,-0.011752,0.935928,0.979824,0.975439,0.053958,0.012391,0.903286,0.963518,0.784563,0.772845,0.014344,0.876400,0.678810,-1.436431,-0.192280,-0.301311,-0.322261,0.010114,0.000346,0.806874,-1.610552,-1.529542,0.071024,0.001630,-0.301439,0.992301,-0.228272,0.020870,0.923492,-0.197112,0.184452
27380,0.926002,0.733766,0.026277,0.889576,0.631396,0.007654,0.826334,0.893561,0.911327,0.085868,0.009466,0.903286,0.963518,0.784563,0.820959,0.001011,0.717949,0.822312,-1.436431,-0.181193,-0.293072,-0.317460,0.087799,0.000077,0.923492,-1.610552,-1.529542,0.020870,0.008685,-0.377830,0.517347,-0.355077,0.000162,0.876400,-0.220258,0.053216
27381,0.206143,-0.034874,-0.015274,0.068401,-0.140084,-0.008599,0.883242,0.717949,0.684162,0.033244,0.007611,0.903286,0.963518,0.784563,0.820959,0.130277,0.876400,0.803590,-1.436431,-0.185906,-0.296578,-0.319505,0.083514,0.000162,0.923492,-1.610552,-1.529542,0.000077,0.030855,-0.310766,0.517347,-0.243432,0.020870,0.893561,-0.199998,0.053216
27382,1.970338,2.558015,0.189885,2.037109,2.522566,0.078694,0.859813,0.876400,0.891423,0.048794,0.003321,0.903286,0.963518,0.784563,0.893561,0.020410,0.717949,0.824122,-1.436431,-0.185164,-0.296027,-0.319184,0.091394,0.020870,0.923492,-1.610552,-1.529542,0.052664,0.017230,-0.266476,0.517347,-0.172201,0.000162,0.820959,-0.186149,0.053216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644661,0.627428,0.377737,0.005013,0.097715,-0.118236,-0.008240,0.866654,0.723644,0.860878,0.091756,0.012826,1.040620,1.175952,0.857113,0.830827,0.260587,0.640807,0.274047,-1.371422,0.823399,0.600116,0.311612,0.041589,0.003185,0.430565,-1.475579,-1.311776,0.001932,0.010048,-0.306770,0.972061,-0.236927,0.014053,0.962602,-0.198764,0.267307
644682,3.336000,7.230545,1.026046,2.997459,5.370732,0.269315,0.887594,0.972061,0.562624,0.104598,0.045703,1.040620,1.175952,0.857113,0.781885,0.026145,0.959197,0.418002,-1.371422,0.765740,0.540092,0.262599,0.007809,0.001646,0.861406,-1.475579,-1.311776,0.029351,0.043282,-0.450410,0.723644,-0.481455,0.020878,0.945406,-0.241221,0.129557
644683,2.141746,2.970728,0.239585,1.607758,1.669413,0.041159,0.877851,0.894355,0.827015,0.076336,0.010921,1.040620,1.175952,0.857113,0.962602,0.032226,0.374614,0.265865,-1.371422,0.797066,0.572556,0.288987,0.045813,0.030675,0.640807,-1.475579,-1.311776,0.001646,0.009939,-0.383200,0.723644,-0.364223,0.036489,0.972061,-0.221843,0.147214
644687,1.573984,1.740293,0.105239,1.041092,0.816925,0.012662,0.784983,0.374614,0.826921,0.191127,0.025323,1.040620,1.175952,0.857113,0.959197,0.041471,0.723644,0.385537,-1.371422,0.806852,0.582769,0.297348,0.023890,0.036489,0.900932,-1.475579,-1.311776,0.223000,0.018678,-0.397351,0.792396,-0.388478,0.003185,0.182226,-0.225994,0.097357


In [ ]:
fc_centroids_bf = pickle.load(open('drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/6.Classification_Performance_on_Household_Indicators/Arch_2/Cluster_Centroids/bf_centroids.pickle','rb'))
fc_centroids_fc = pickle.load(open('drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/6.Classification_Performance_on_Household_Indicators/Arch_2/Cluster_Centroids/fc_centroids.pickle','rb'))
fc_centroids_msw = pickle.load(open('drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/6.Classification_Performance_on_Household_Indicators/Arch_2/Cluster_Centroids/msw_centroids.pickle','rb'))
fc_centroids_asset = pickle.load(open('drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/6.Classification_Performance_on_Household_Indicators/Arch_2/Cluster_Centroids/asset_centroids.pickle','rb'))
fc_centroids_lit = pickle.load(open('drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/6.Classification_Performance_on_Household_Indicators/Arch_2/Cluster_Centroids/lit_centroids.pickle',''))


fc_centroids_bf = np.array(list(fc_centroids_bf.values()))rb
fc_centroids_fc = np.array(list(fc_centroids_fc.values()))
fc_centroids_msw = np.array(list(fc_centroids_msw.values()))
fc_centroids_asset = np.array(list(fc_centroids_asset.values()))
fc_centroids_lit = np.array(list(fc_centroids_lit.values()))

print(fc_centroids_bf)
print(fc_centroids_asset)
print(fc_centroids_lit)

[[95.06597392  0.48478858  4.25194301]
 [28.43290845 68.42038732  2.83851408]
 [32.05076422  1.07754624 66.01605086]]
[[ 8.42653522 20.6603894   5.16657173  0.39988889]
 [22.19590974 63.17948254 12.93166811  1.49625408]
 [62.26585789 75.13256883 30.8741908   3.59045989]]
[[34.48335181]
 [53.03561535]
 [68.6060283 ]]


In [ ]:
clf_rud_bf = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/bf_rud_reg.pickle','rb'))
clf_int_bf = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/bf_int_reg.pickle','rb'))
clf_adv_bf = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/bf_adv_reg.pickle','rb'))

clf_rud_fc = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/fc_rud_reg.pickle','rb'))
clf_int_fc = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/fc_int_reg.pickle','rb'))
clf_adv_fc = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/fc_adv_reg.pickle','rb'))

clf_rud_msw = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/msw_rud_reg.pickle','rb'))
clf_int_msw = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/msw_int_reg.pickle','rb'))
clf_adv_msw = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/msw_adv_reg.pickle','rb'))

clf_tv = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/tv_reg.pickle','rb'))
clf_tele = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/tele_reg.pickle','rb'))
clf_2w = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/2w_reg.pickle','rb'))
clf_4w = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/4w_reg.pickle','rb'))

clf_lit = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Models/2011_Household_Indicators_Regression/viirs_models/lit_reg.pickle','rb'))


In [ ]:
rud_bf, inter_bf, adv_bf = clf_rud_bf.predict(X11), clf_int_bf.predict(X11), clf_adv_bf.predict(X11)
rud_fc, inter_fc, adv_fc = clf_rud_fc.predict(X11), clf_int_fc.predict(X11), clf_adv_fc.predict(X11)
rud_msw, inter_msw, adv_msw = clf_rud_msw.predict(X11), clf_int_msw.predict(X11), clf_adv_msw.predict(X11)

tv11, tele11, twow11, fourw11 = clf_tv.predict(X11), clf_tele.predict(X11), clf_2w.predict(X11), clf_4w.predict(X11)
lit11 = clf_lit.predict(X11)


In [ ]:
pred_level_bf, pred_level_fc, pred_level_msw, pred_level_asset, pred_level_lit = [], [], [], [], []
# pred_level_11 = []

for i in range(len(lit11)):
  lvl_bf = np.argmin(np.linalg.norm(np.array([rud_bf[i], inter_bf[i], adv_bf[i]])-fc_centroids_bf, ord=2, axis=1))+1
  lvl_fc = np.argmin(np.linalg.norm(np.array([rud_fc[i], inter_fc[i], adv_fc[i]])-fc_centroids_fc, ord=2, axis=1))+1
  lvl_msw = np.argmin(np.linalg.norm(np.array([rud_msw[i], inter_msw[i], adv_msw[i]])-fc_centroids_msw, ord=2, axis=1))+1

  lvl_asset = np.argmin(np.linalg.norm(np.array([tv11[i], tele11[i], twow11[i], fourw11[i]])-fc_centroids_asset, ord=2, axis=1))+1

  lvl_lit = np.argmin(np.linalg.norm(np.array([lit11[i]])-fc_centroids_lit, ord=2, axis=1))+1


  # lvl01 = np.argmin(np.linalg.norm(np.array([rud01[i], inter01[i], adv01[i]])-fc_centroids, ord=2, axis=1))+1
  # lvl11 = np.argmin(np.linalg.norm(np.array([tv11[i], tele11[i], twow11[i], fourw11[i]])-fc_centroids, ord=2, axis=1))+1
  # lvl01 = np.argmin(np.linalg.norm(np.array([tv01[i], tele01[i], twow01[i], fourw01[i]])-fc_centroids, ord=2, axis=1))+1
  pred_level_bf += [lvl_bf]
  pred_level_fc += [lvl_fc]
  pred_level_msw += [lvl_msw]
  pred_level_asset += [lvl_asset]
  pred_level_lit += [lvl_lit]
  # pred_level_01 += [lvl01]

In [ ]:
OUT = IN.copy()
OUT["BF"] = pred_level_bf
OUT["FC"] = pred_level_fc
OUT["MSW"] = pred_level_msw
OUT["ASSET"] = pred_level_asset
OUT["LIT"] = pred_level_lit

In [ ]:
OUT = OUT[["BF","FC","MSW","ASSET","LIT"]]

In [ ]:
OUT["ADI"] = OUT["BF"] + OUT["FC"] + OUT["MSW"] + OUT["ASSET"] + OUT["LIT"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
OUT.describe()

,BF,FC,MSW,ASSET,LIT,ADI
count,309525.000000,309525.000000,309525.000000,309525.000000,309525.000000,309525.000000
mean,1.064838,1.540767,2.016971,1.968381,2.124782,8.715739
std,0.352314,0.779636,0.667852,0.506167,0.388452,1.812381
min,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
25%,1.000000,1.000000,2.000000,2.000000,2.000000,8.000000
50%,1.000000,1.000000,2.000000,2.000000,2.000000,9.000000
75%,1.000000,2.000000,2.000000,2.000000,2.000000,9.000000
max,3.000000,3.000000,3.000000,3.000000,3.000000,15.000000


In [ ]:
OUT.head()

,BF,FC,MSW,ASSET,LIT,ADI
27378,1,3,3,3,3,13
27379,1,3,3,3,3,13
27380,1,3,3,3,3,13
27381,1,1,3,3,3,11
27382,1,3,3,3,3,13


In [ ]:
save_as = "pred_"+year+".pkl"
pickle.dump(OUT, open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/"+save_as,"wb"))

Testing and Merging Phase

In [ ]:
adi_1999 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_1999.pkl")
adi_2000 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2000.pkl")
adi_2001 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2001.pkl")

adi_2009 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2009.pkl")
adi_2010 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2010.pkl")
adi_2011 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2011.pkl")

adi_2017 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2017.pkl")
adi_2018 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2018.pkl")
adi_2019 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2019.pkl")


In [ ]:
def adi_dict(df):
  dict_adi = {}
  for index, row in df.iterrows():
    dict_adi[index] = {"BF": row["BF"],
                       "FC": row["FC"],
                       "MSW": row["MSW"],
                       "ASSET": row["ASSET"],
                       "LIT": row["LIT"],
                       "ADI":row["ADI"]}
  return dict_adi

In [ ]:
dict_1999 = adi_dict(adi_1999)
dict_2000 = adi_dict(adi_2000)
dict_2001 = adi_dict(adi_2001)

dict_2009 = adi_dict(adi_2009)
dict_2010 = adi_dict(adi_2010)
dict_2011 = adi_dict(adi_2011)

dict_2017 = adi_dict(adi_2017)
dict_2018 = adi_dict(adi_2018)
dict_2019 = adi_dict(adi_2019)

In [ ]:
gt_asset = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/5.ADI_Analysis/2011_Original(Ground_Truth)_ADI_without_outlier/2011_ADI_orginal_asset_without_outlier.pickle", "rb"))

gt_msw = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/5.ADI_Analysis/2011_Original(Ground_Truth)_ADI_without_outlier/2011_ADI_orginal_msw_without_outlier.pickle", "rb"))

gt_fc = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/5.ADI_Analysis/2011_Original(Ground_Truth)_ADI_without_outlier/2011_ADI_orginal_fc_without_outlier.pickle", "rb"))

gt_lit = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/5.ADI_Analysis/2011_Original(Ground_Truth)_ADI_without_outlier/2011_ADI_orginal_lit_without_outlier.pickle", "rb"))

gt_bf = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/5.ADI_Analysis/2011_Original(Ground_Truth)_ADI_without_outlier/2011_ADI_orginal_bf_without_outlier.pickle", "rb"))



In [ ]:
dict_gt = {}
for i in gt_asset["Town/Village(Code)"]:
  dict_gt[i] = {}
for index,row in gt_asset.iterrows():
  dict_gt[row['Town/Village(Code)']]["ASSET"] = row['ASSET_ADI']
for index,row in gt_fc.iterrows():
  dict_gt[row['Town/Village(Code)']]["FC"] = row['FC_ADI']
for index,row in gt_bf.iterrows():
  dict_gt[row['Town/Village(Code)']]["BF"] = row['BF_ADI']
for index,row in gt_msw.iterrows():
  dict_gt[row['Town/Village(Code)']]["MSW"] = row['MSW_ADI']
for index,row in gt_lit.iterrows():
  dict_gt[row['Town/Village(Code)']]["LIT"] = row['Label']

In [ ]:
gt_lit.columns

Index(['State', 'District', 'Subdistt', 'Town/Village(Code)', 'Ward', 'Name',
       'literacy_percent_2011', 'Label', 'State_name'],
      dtype='object')

In [ ]:
for i in dict_gt:
  dict_gt[i]["ADI"] = 0
  for j in ["ASSET", "FC", "BF", "MSW", "LIT"]:
    dict_gt[i]["ADI"] += dict_gt[i][j]


In [ ]:
dut = {}
for i in dict_gt:
  if len(dict_gt[i]) not in dut:
    dut[len(dict_gt[i])] = 1
  else:
    dut[len(dict_gt[i])] += 1
print(dut)
  

{6: 340953}


In [ ]:
from sklearn.metrics import classification_report
for j in ["ASSET", "FC", "BF", "MSW", "LIT"]:
  print("Label = ", j)
  gt = []
  pred = []
  for i in dict_gt:
    if i in dict_2011:
      gt.append(dict_gt[i][j])
      pred.append(dict_2011[i][j])
  print(len(gt))
  print(classification_report(gt, pred))

Label =  ASSET
340953
              precision    recall  f1-score   support

           1       0.84      0.70      0.76    120724
           2       0.70      0.84      0.76    154166
           3       0.82      0.67      0.74     66063

    accuracy                           0.76    340953
   macro avg       0.79      0.74      0.76    340953
weighted avg       0.77      0.76      0.76    340953

Label =  FC
340953
              precision    recall  f1-score   support

           1       0.85      0.91      0.88    205449
           2       0.70      0.75      0.73     69234
           3       0.55      0.40      0.46     66270

    accuracy                           0.78    340953
   macro avg       0.70      0.68      0.69    340953
weighted avg       0.76      0.78      0.76    340953

Label =  BF
340953
              precision    recall  f1-score   support

           1       0.88      0.98      0.93    271778
           2       0.76      0.31      0.44     21739
           3   

In [ ]:
gt = []
pred = []
from sklearn.metrics import mean_squared_error
import math
for i in dict_gt:
  if i in dict_2011:
    gt.append(dict_gt[i]["ADI"])
    pred.append(dict_2011[i]["ADI"])

mse = mean_squared_error(gt, pred)

rmse = math.sqrt(mse)

print(rmse)

1.530495572267186


In [ ]:
gt = []
pred = []
for i in dict_gt:
  if i in final_adi_2011:
    gt.append(dict_gt[i]["ADI"])
    pred.append(final_adi_2011[i])

mse = mean_squared_error(gt, pred)

rmse = math.sqrt(mse)

print(rmse)

1.7522556123035522


In [ ]:
ind_label_final_2011 = {}
for i in dict_2011:
  if i in dict_2009 and i in dict_2010:
    ind_label_final_2011[i] = {}
    for j in ["ASSET", "FC", "BF", "MSW", "LIT"]:
      # x = np.array([1, 2, 3])
      y = int( (dict_2009[i][j]+ dict_2010[i][j]+ dict_2011[i][j])/3)
      # m, b = np.polyfit(x, y, 1)
      ind_label_final_2011[i][j] = y

In [ ]:
for i in ind_label_final_2011:
  ind_label_final_2011[i]["ADI"] = 0
  for j in ["ASSET", "FC", "BF", "MSW", "LIT"]:
    ind_label_final_2011[i]["ADI"] += ind_label_final_2011[i][j]

In [ ]:
for j in ["ASSET", "FC", "BF", "MSW", "LIT"]:
  print("Label = ", j)
  gt = []
  pred = []
  for i in dict_gt:
    if i in ind_label_final_2011:
      gt.append(dict_gt[i][j])
      pred.append(ind_label_final_2011[i][j])
  print(len(gt))
  print(classification_report(gt, pred))

Label =  ASSET
332899
              precision    recall  f1-score   support

           1       0.75      0.73      0.74    119800
           2       0.60      0.82      0.69    150455
           3       0.89      0.17      0.29     62644

    accuracy                           0.67    332899
   macro avg       0.75      0.57      0.58    332899
weighted avg       0.71      0.67      0.64    332899

Label =  FC
332899
              precision    recall  f1-score   support

           1       0.73      0.92      0.82    199051
           2       0.50      0.53      0.51     69159
           3       0.66      0.08      0.14     64689

    accuracy                           0.68    332899
   macro avg       0.63      0.51      0.49    332899
weighted avg       0.67      0.68      0.62    332899

Label =  BF
332899
              precision    recall  f1-score   support

           1       0.82      0.99      0.90    265441
           2       0.25      0.11      0.15     21412
           3   

In [ ]:
gt = []
pred = []
for i in dict_gt:
  if i in ind_label_final_2011:
    gt.append(dict_gt[i]["ADI"])
    pred.append(ind_label_final_2011[i]["ADI"])

mse = mean_squared_error(gt, pred)

rmse = math.sqrt(mse)

print(rmse)

2.214994984695279


In [ ]:
x = np.array([1, 2, 3])
y = np.array([2,1,1])
m, b = np.polyfit(x, y, 1)
int(m*3+b)

0

In [ ]:
final_adi_2001 = {}
for i in dict_1999:
  if i in dict_2000 and i in dict_2001:
    final_adi_2001[i] = {}
    x = np.array([1, 2, 3])
    y = np.array([ dict_1999[i]["ADI"], dict_2000[i]["ADI"], dict_2001[i]["ADI"]])
    m, b = np.polyfit(x, y, 1)
    final_adi_2001[i]["ADI"] = int(m*3+b)
    for j in dict_2001[i]:
      if j != "ADI":
        final_adi_2001[i][j] = dict_2001[i][j]

In [ ]:
final_adi_2011 = {}
for i in dict_2009:
  if i in dict_2010 and i in dict_2011:
    final_adi_2011[i] = {}
    x = np.array([1, 2, 3])
    y = np.array([ dict_2009[i]["ADI"], dict_2010[i]["ADI"], dict_2011[i]["ADI"]])
    m, b = np.polyfit(x, y, 1)
    final_adi_2011[i]["ADI"] = int(m*3+b)
    for j in dict_2011[i]:
      if j != "ADI":
        final_adi_2011[i][j] = dict_2011[i][j]    


In [ ]:
final_adi_2019 = {}
for i in dict_2019:
  if i in dict_2017 and i in dict_2018:
    final_adi_2019[i] = {}
    x = np.array([1, 2, 3])
    y = np.array([ dict_2017[i]["ADI"], dict_2018[i]["ADI"], dict_2019[i]["ADI"]])
    m, b = np.polyfit(x, y, 1)
    final_adi_2019[i]["ADI"] = int(m*3+b)
    for j in dict_2019[i]:
      if j != "ADI":
        final_adi_2019[i][j] = dict_2019[i][j]    

In [ ]:
dut = {}
for i in final_adi_2001:
  if len(final_adi_2001[i]) not in dut:
    dut[len(final_adi_2001[i])] = 1
  else:
    dut[len(final_adi_2001[i])] += 1
print(dut)
  

{6: 291078}


In [ ]:
count_common = 0
count_dec = 0
for i in final_adi_2019:
  if i in final_adi_2011:
    count_common += 1
    if (final_adi_2011[i] > final_adi_2019[i]):
      count_dec += 1
print("Common", count_common)
print("Decreased", count_dec)
print(100*count_dec/count_common, "%")

Common 308708
Decreased 116255
37.65856407997201 %


In [ ]:
count_common = 0
count_dec = 0
for i in dict_2019:
  if i in dict_2011:
    count_common += 1
    if (dict_2011[i]["ADI"] > dict_2019[i]["ADI"]):
      count_dec += 1
print("Common", count_common)
print("Decreased", count_dec)
print(100*count_dec/count_common, "%")

Common 309525
Decreased 105168
33.97722316452629 %


In [ ]:
count_common = 0
count_dec = 0
for i in final_adi_2001:
  if i in final_adi_2011:
    count_common += 1
    if (final_adi_2001[i] > final_adi_2011[i]):
      count_dec += 1
print("Common", count_common)
print("Decreased", count_dec)
print(100*count_dec/count_common, "%")

Common 290269
Decreased 79799
27.491395912067773 %


In [ ]:
count_common = 0
count_dec = 0
for i in dict_2001:
  if i in dict_2011:
    count_common += 1
    if (dict_2001[i]["ADI"] > dict_2011[i]["ADI"]):
      count_dec += 1
print("Common", count_common)
print("Decreased", count_dec)
print(100*count_dec/count_common, "%")

Common 311831
Decreased 89213
28.6094070185453 %


In [ ]:
d = {"village_code": [], "ADI":[], "BF":[], "FC":[], "ASSET":[], "LIT":[], "MSW":[]}
for i in dict_gt:
  d["village_code"].append(i)
  for j in dict_gt[i]:
    d[j].append(dict_gt[i][j])

ground_truth_2011 = pd.DataFrame(data=d)
save_as = "ground_truth_2011.pkl"
pickle.dump(ground_truth_2011, open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/"+save_as,"wb"))

In [ ]:
ground_truth_2011.head()

,village_code,ADI,BF,FC,ASSET,LIT,MSW
0,34223,11,3,1,3,2,2
1,34224,14,3,3,3,3,2
2,34225,13,2,3,3,2,3
3,34228,12,1,3,3,2,3
4,34230,12,3,2,3,2,2


In [ ]:
d = {"village_code": [], "ADI":[], "BF":[], "FC":[], "ASSET":[], "LIT":[], "MSW":[]}
for i in final_adi_2019:
  d["village_code"].append(i)
  for j in final_adi_2019[i]:
    d[j].append(final_adi_2019[i][j])

df_final_2019 = pd.DataFrame(data=d)
save_as = "pred_final_2019.pkl"
pickle.dump(df_final_2019, open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/"+save_as,"wb"))

In [ ]:
df_final_2019.head()

,village_code,ADI,BF,FC,ASSET,LIT,MSW
0,27378,12,1,3,3,3,3
1,27379,12,1,3,3,3,3
2,27380,12,1,3,3,3,3
3,27381,10,1,1,3,3,3
4,27382,12,1,3,3,3,3


In [ ]:
d = {"village_code": [], "ADI":[], "BF":[], "FC":[], "ASSET":[], "LIT":[], "MSW":[]}
for i in final_adi_2011:
  d["village_code"].append(i)
  for j in final_adi_2011[i]:
    d[j].append(final_adi_2011[i][j])

df_final_2011 = pd.DataFrame(data=d)
save_as = "pred_final_2011.pkl"
pickle.dump(df_final_2011, open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/"+save_as,"wb"))

In [ ]:
d = {"village_code": [], "ADI":[], "BF":[], "FC":[], "ASSET":[], "LIT":[], "MSW":[]}
for i in final_adi_2001:
  d["village_code"].append(i)
  for j in final_adi_2001[i]:
    d[j].append(final_adi_2001[i][j])
df_final_2001 = pd.DataFrame(data=d)
save_as = "pred_final_2001.pkl"
pickle.dump(df_final_2001, open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/"+save_as,"wb"))

In [ ]:
df_final_2001.head()

,village_code,ADI,BF,FC,ASSET,LIT,MSW
0,27378,6,1,1,2,2,1
1,27379,7,1,1,2,2,1
2,27380,7,1,1,2,2,1
3,27381,6,1,1,2,2,1
4,27382,6,1,1,2,2,1


In [ ]:
df_final_2011.head()

,village_code,ADI,BF,FC,ASSET,LIT,MSW
0,27378,11,1,1,3,3,3
1,27379,13,1,3,3,3,3
2,27380,11,1,1,3,3,3
3,27381,11,1,1,3,3,3
4,27382,11,1,1,3,3,3


In [ ]:
df_final_2019.head()

,village_code,ADI,BF,FC,ASSET,LIT,MSW
0,27378,12,1,3,3,3,3
1,27379,12,1,3,3,3,3
2,27380,12,1,3,3,3,3
3,27381,10,1,1,3,3,3
4,27382,12,1,3,3,3,3


In [ ]:
df_final_2001.tail()

,village_code,ADI
291073,644661,9
291074,644682,8
291075,644683,10
291076,644687,9
291077,644693,8


In [ ]:
final_adi_2001[644687]

9

In [ ]:
df_final_2019 = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/pred_final_2019.pkl","rb"))

In [ ]:
df_final_2011 = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/pred_final_2011.pkl","rb"))

In [ ]:
df_final_2001 = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/pred_final_2001.pkl","rb"))

In [ ]:
df_final_gt_2011 = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/ground_truth_2011.pkl", "rb"))

In [ ]:
final_2019 = adi_dict(df_final_2019)
final_2011 = adi_dict(df_final_2011)
final_2001 = adi_dict(df_final_2001)


In [ ]:
gt_2011_dict = adi_dict(df_final_gt_2011)

In [ ]:
total_vil = 0
dec = 0
for i in gt_2011_dict:
  if i in final_2019:
    total_vil += 1
    if gt_2011_dict[i]["ADI"] > final_2019[i]["ADI"]:
      dec += 1
print(dec, total_vil)
print("Decrement = ", 100*dec/total_vil, "%")

166274 309519
Decrement =  53.72012703582009 %


In [ ]:
total_vil = 0
dec = 0
for i in gt_2011_dict:
  if i in final_2019:
    total_vil += 1
    if gt_2011_dict[i]["ADI"] > final_2019[i]["ADI"]:
      dec += 1
print(dec, total_vil)
print("Decrement = ", 100*dec/total_vil, "%")

164716 309519
Decrement =  53.216765368200335 %


In [ ]:
for i in final_2019:
  final_2019[i]["ADI"] -= final_2019[i]["LIT"]

In [ ]:
for i in gt_2011_dict:
  gt_2011_dict[i]["ADI"] -= gt_2011_dict[i]["LIT"]

In [ ]:
adi_1999 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_1999.pkl")
adi_2000 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2000.pkl")
adi_2001 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2001.pkl")

adi_2009 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2009.pkl")
adi_2010 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2010.pkl")
adi_2011 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2011.pkl")

adi_2017 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2017.pkl")
adi_2018 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2018.pkl")
adi_2019 = pd.read_pickle("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/ADI_Predictions/pred_2019.pkl")

In [ ]:
df_final_gt_2011 = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/ground_truth_2011.pkl", "rb"))

In [ ]:
adi_2019.head()

,BF,FC,MSW,ASSET,LIT,ADI
27378,1,3,3,3,3,13
27379,1,3,3,3,3,13
27380,1,3,3,3,3,13
27381,1,1,3,3,3,11
27382,1,3,3,3,3,13


In [ ]:
df_final_gt_2011 = df_final_gt_2011.set_index('village_code')

In [ ]:
df_final_gt_2011.columns

Index(['ADI', 'BF', 'FC', 'ASSET', 'LIT', 'MSW'], dtype='object')

In [ ]:
adi_2019.columns

Index(['BF', 'FC', 'MSW', 'ASSET', 'LIT', 'ADI'], dtype='object')

In [ ]:
adi_2019.rename(columns = {'ADI':'ADI_pred_2019'}, inplace = True)

In [ ]:
adi_2019["ADI_pred_2019"] = adi_2019["ADI_pred_2019"] - adi_2019["LIT"]

In [ ]:
gt_pred_join = df_final_gt_2011[["ADI_GT_2011"]].join(adi_2019[["ADI_pred_2019"]], how="inner")

In [ ]:
gt_pred_join.shape

(139487, 2)

In [ ]:
gt_pred_join["Diff"] = gt_pred_join["ADI_pred_2019"] - gt_pred_join["ADI_GT_2011"]

In [ ]:
gt_pred_join[gt_pred_join['Diff'] < 0]

,ADI_GT_2011,ADI_pred_2019,Diff
34224,14,11,-3
34225,13,11,-2
34230,12,10,-2
34234,11,8,-3
34235,10,9,-1
...,...,...,...
632170,11,8,-3
632171,15,8,-7
632199,13,7,-6
632200,13,9,-4


In [ ]:
print(132462/309525)

0.4279525078749697


In [ ]:
df_final_2019 = pickle.load(open("/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Outputs/Time_Series_Output/pred_final_2019.pkl","rb"))

In [ ]:
df_final_2019.rename(columns = {'ADI':'ADI_pred_2019'}, inplace = True)

In [ ]:
df_final_2019["ADI_pred_2019"] = df_final_2019["ADI_pred_2019"] - df_final_2019["LIT"]

In [ ]:
df_final_gt_2011.rename(columns = {'ADI':'ADI_GT_2011'}, inplace = True)

In [ ]:
df_final_gt_2011.columns

Index(['village_code', 'ADI_GT_2011', 'BF', 'FC', 'ASSET', 'LIT', 'MSW'], dtype='object')

In [ ]:
df_final_gt_2011['ADI_GT_2011'] = df_final_gt_2011['ADI_GT_2011'] - df_final_gt_2011["LIT"]

In [ ]:
gt_pred_join_corrected = df_final_gt_2011[["ADI_GT_2011"]].join(df_final_2019[["ADI_pred_2019"]], how="inner")

In [ ]:
gt_pred_join_corrected.shape

(309519, 2)

In [ ]:
gt_pred_join_corrected["Diff"] = gt_pred_join_corrected["ADI_pred_2019"] - gt_pred_join_corrected["ADI_GT_2011"]

In [ ]:
gt_pred_join_corrected[gt_pred_join_corrected['Diff'] < 0]

,ADI_GT_2011,ADI_pred_2019,Diff
1,11,9,-2
2,11,9,-2
3,10,7,-3
4,10,9,-1
6,8,7,-1
...,...,...,...
309485,7,5,-2
309486,7,6,-1
309487,8,6,-2
309488,8,7,-1


In [ ]:
(164716/309519)*100

53.216765368200335

In [ ]:
adi_2017.describe()

,BF,FC,MSW,ASSET,LIT,ADI
count,309530.000000,309530.000000,309530.000000,309530.000000,309530.000000,309530.000000
mean,1.072400,1.531548,2.035199,1.916874,2.152509,8.708529
std,0.371407,0.776810,0.639249,0.550072,0.427426,1.865154
min,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
25%,1.000000,1.000000,2.000000,2.000000,2.000000,7.000000
50%,1.000000,1.000000,2.000000,2.000000,2.000000,9.000000
75%,1.000000,2.000000,2.000000,2.000000,2.000000,10.000000
max,3.000000,3.000000,3.000000,3.000000,3.000000,15.000000


In [ ]:
adi_2018.describe()

,BF,FC,MSW,ASSET,LIT,ADI
count,309529.000000,309529.000000,309529.000000,309529.000000,309529.000000,309529.000000
mean,1.065383,1.546430,2.010203,1.919513,2.137357,8.678886
std,0.353552,0.781325,0.637489,0.540776,0.420028,1.850127
min,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
25%,1.000000,1.000000,2.000000,2.000000,2.000000,7.000000
50%,1.000000,1.000000,2.000000,2.000000,2.000000,9.000000
75%,1.000000,2.000000,2.000000,2.000000,2.000000,10.000000
max,3.000000,3.000000,3.000000,3.000000,3.000000,15.000000


In [ ]:
adi_2019.describe()

,BF,FC,MSW,ASSET,LIT,ADI
count,309525.000000,309525.000000,309525.000000,309525.000000,309525.000000,309525.000000
mean,1.064838,1.540767,2.016971,1.968381,2.124782,8.715739
std,0.352314,0.779636,0.667852,0.506167,0.388452,1.812381
min,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
25%,1.000000,1.000000,2.000000,2.000000,2.000000,8.000000
50%,1.000000,1.000000,2.000000,2.000000,2.000000,9.000000
75%,1.000000,2.000000,2.000000,2.000000,2.000000,9.000000
max,3.000000,3.000000,3.000000,3.000000,3.000000,15.000000


In [ ]:
df_final_gt_2011.describe()

,village_code,ADI_GT_2011,BF,FC,ASSET,LIT,MSW
count,340953.000000,340953.000000,340953.000000,340953.000000,340953.000000,340953.000000,340953.000000
mean,346753.937554,9.015709,1.342015,1.591794,1.839682,2.199177,2.043041
std,183608.510687,2.328290,0.709435,0.793920,0.722591,0.706231,0.649149
min,27378.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,166727.000000,7.000000,1.000000,1.000000,1.000000,2.000000,2.000000
50%,393617.000000,9.000000,1.000000,1.000000,2.000000,2.000000,2.000000
75%,505255.000000,10.000000,1.000000,2.000000,2.000000,3.000000,2.000000
max,644693.000000,15.000000,3.000000,3.000000,3.000000,3.000000,3.000000


In [ ]:
gt_pred_join_corrected["ADI_pred_2019"] - gt_pred_join_corrected["ADI_GT_2011"]